# Demo No. 12 - Light curve fitting.

This demo will guide you trough a process of inferring a parameters of the eclipsing binary from the shape of the light curve. We will stat again by importing necessary modules and setting our loging:

In [1]:
import os

import astropy.units as u

from elisa.conf import config
from elisa.analytics import LCData
from elisa.analytics import BinarySystemAnalyticsTask


# setting up a custom logging config to prevent unreasonably long log messages during fit
config.LOG_CONFIG ='jupyter_fit_logging.json'
config.set_up_logging() 

In this particular case of KIC 4851217, we will help ourselves by using previous results obtained from radial velocity fit. In particular, radial velocities are much more sensitive to mass ratio of the components, therefore we will adopt the value obtained in previous demo and we will keep it fixed. 

The procedure itself is very similar to the one mentioned in the previous demo 11. Initially, we will initiate the dataset with our Kepler phase curve:

In [2]:
kepler_data = LCData.load_from_file(filename='demo_data/lc_data/kepler_phs_crv.dat', 
                                    x_unit=u.dimensionless_unscaled, 
                                    y_unit=u.dimensionless_unscaled
                                   )

This will be followed by initialization of `BinarySystemAnalyticsTask`, where light curves are provided as values of dictionary with filter names as their corresponding keys:

In [3]:
task = BinarySystemAnalyticsTask(light_curves={'Kepler': kepler_data})

Similarly to rv_fit module, we can also pick from `standard` and `community` approach. We can acces lists of available parameters in a similar fashion:

In [5]:
task.lc_fit.FIT_PARAMS_COMBINATIONS

{'standard': ['p__mass',
  's__mass',
  'inclination',
  'eccentricity',
  'argument_of_periastron',
  'period',
  'primary_minimum_time',
  'p__t_eff',
  's__t_eff',
  'p__surface_potential',
  's__surface_potential',
  'p__gravity_darkening',
  's__gravity_darkening',
  'p__albedo',
  's__albedo',
  'additional_light',
  'phase_shift'],
 'community': ['mass_ratio',
  'semi_major_axis',
  'inclination',
  'eccentricity',
  'argument_of_periastron',
  'period',
  'primary_minimum_time',
  'p__t_eff',
  's__t_eff',
  'p__surface_potential',
  's__surface_potential',
  'p__gravity_darkening',
  's__gravity_darkening',
  'p__albedo',
  's__albedo',
  'additional_light',
  'phase_shift']}

Using this dictionary, we can define our starting parameters:

In [ ]:
lc_initial = {
    'inclination': {
        'value': 81.0,
        'fixed': False,
        'min': 79,
        'max': 83
    },
    'eccentricity': {
        'value': 0.01,
        'fixed': False,
        'min': 0.00,
        'max': 0.05,
    },  # value obtained during RV fit but kept variable due to poor coverage of RV measurements and sensitivity of LC 
        # on eccentricity
    'mass_ratio': {
        'value': 1.077,
        'fixed': True,
    },  # value obtained during RV fit
    'argument_of_periastron': {
        'value': 285,
        'fixed': False,
        'min': 0,
        'max': 360,
    },  # similar case to eccentricity
    'p__t_eff': {
        'value': 7022.0,
        'fixed': True,
    },  # fixed to value found in literature
    's__t_eff': {
        'value': 6961.75240,
        'fixed': False,
        'min': 6700.0,
        'max': 7300.0
    },
    'p__surface_potential': {
        'value': 8.1,
        'fixed': False,
        'min': 7.6,
        'max': 8.7,
    },
    's__surface_potential': {
        'value': 5.2,
        'fixed': False,
        'min': 4.8,
        'max': 5.8,
    },
    'p__albedo': {
        'value': 1.0,
        'fixed': True
    },  
    's__albedo': {
        'value': 1.0,
        'fixed': True
    },
    'p__gravity_darkening': {
        'value': 1.0,
        'fixed': True
    },  
    's__gravity_darkening': {
        'value': 1.0,
        'fixed': True
    },  # we are presuming radiative envelopes on both components
    'semi_major_axis': {
        'constraint': '11.86 / sin(radians({inclination}))'
    },  # using parameter asin(i) obtained from RV fit
    'period': {
        'value': 2.47028376,
        'fixed': True,
    },
    'additional_light': {
        'value': 0.2415143632,
        'fixed': False,
        'min': 0.20,
        'max': 0.30,
    },  # there is strong evidence for the presence of the third body
    'phase_shift': {
        'value': 0,
        'fixed': False,
        'min': -0.02,
        'max': 0.02,
    },  # accounting for missalignment of the phase curve
}